# LOG6302A — Analyse d’applications et Cyber-sécurité<br>Laboratoire #5

**Quentin Guidée (2206809), Nam Vu (2230468)**

Polytechnique Montréal – Hiver 2024


In [ ]:
import tempfile
from collections import defaultdict
from collections.abc import Iterable
from copy import deepcopy
from typing import NamedTuple, Self

from code_analysis import CFG, ASTReader, CFGReader, Graph
from graphviz import Source
from IPython.display import Image
from pydantic import BaseModel

In [ ]:
ast_reader = ASTReader()
cfg_reader = CFGReader()


def show_graph(graph: Graph):
    """Afficher le graphe dans Jupyter"""
    dot = graph.to_dot()
    s = Source(dot)
    with tempfile.NamedTemporaryFile(suffix=".png") as f:
        s.render(f.name, format="png")
        display(Image(f.name + ".png"))


def show_in_out(
    cfg: CFG,
    in_dict: dict[int, set[int]],
    out_dict: dict[int, set[int]],
) -> CFG:
    """Annote un CFG avec les valeurs de in et out pour chaque noeud puis l'affiche"""
    cfg_copy = deepcopy(cfg)
    for nid in cfg_copy.get_node_ids():
        cfg_copy.set_image(
            nid,
            f"in={[cfg.get_image(v) for v in in_dict[nid]]}\n"
            f"out={[cfg.get_image(v) for v in out_dict[nid]]}\n"
            f"{cfg_copy.get_image(nid) or ''}",
        )
    show_graph(cfg_copy)
    return cfg_copy

In [ ]:
class Pair(NamedTuple):
    def_nid: int
    ref_nid: int


class Taint(BaseModel):
    defs: list[int]
    refs: list[int]
    pairs: list[Pair]
    sinks: list[int]
    filters: list[int]
    safes: list[int]
    sources: list[int]

    @classmethod
    def from_json(cls, taint_file: str) -> Self:
        with open(taint_file, "r") as f:
            taint = cls.model_validate_json(f.read())
        return taint

In [ ]:
def get_key(cfg: CFG, nid: int) -> tuple[str, str]:
    return (cfg.get_var_scope(nid), cfg.get_var_id(nid))


class PossiblyTaintedDefinition:
    def __init__(self):
        self.cfg: CFG
        self.taint: Taint

        self.in_dict: dict[int, set[int]]
        self.out_dict: dict[int, set[int]]

        self.visited: set[int]
        self.worklist: list[int]

    def get_gen(self, nid: int) -> set[int]:
        gen_set = set[int]()
        for nid in self.taint.sinks:
            for definition, reference in self.taint.pairs:
                if reference == nid and definition in self.in_dict[nid]:
                    gen_set.add(definition)
        for nid in self.taint.sources:
            gen_set.add(nid)
        return gen_set

    def get_kill(self, nid: int) -> set[int]:
        kill_set = set[int]()
        if nid in self.taint.defs:
            nid_key = get_key(self.cfg, nid)
            for other_nid in self.taint.defs:
                other_nid_key = get_key(self.cfg, other_nid)
                if nid_key == other_nid_key:
                    kill_set.add(other_nid)
        return kill_set

    def pre_loop_init(self) -> Iterable[None]:
        for entry_nid in self.get_entry_node():
            self.in_dict[entry_nid] = set()
            self.visited.add(entry_nid)
            self.worklist.append(entry_nid)
            yield

    def get_entry_node(self) -> Iterable[int]:
        node_ids = self.cfg.get_node_ids()
        for nid in node_ids:
            if self.cfg.get_type(nid) == "Entry":
                yield nid

    def apply_flow_eq(self, nid: int) -> None:
        self.out_dict[nid] = self.get_gen(nid) | (
            self.in_dict[nid] - self.get_kill(nid)
        )

    def next_nodes(self, nid: int) -> Iterable[int]:
        return self.cfg.get_any_children(nid)

    def can_propagate(self, nid: int, next_nid: int) -> bool:
        return (self.out_dict[nid] - self.in_dict[next_nid]) != set()

    def propagate(self, nid: int, next_nid: int) -> None:
        self.in_dict[next_nid] |= self.out_dict[nid]

    def __call__(
        self, cfg: CFG, taint: Taint
    ) -> tuple[dict[int, set[int]], dict[int, set[int]]]:
        self.cfg = cfg
        self.taint = taint

        self.in_dict = defaultdict(set)
        self.out_dict = defaultdict(set)

        self.visited = set()
        self.worklist = []
        for _ in self.pre_loop_init():
            while self.worklist:
                nid = self.worklist.pop()
                self.apply_flow_eq(nid)
                for next_nid in self.next_nodes(nid):
                    if next_nid not in self.visited or self.can_propagate(
                        nid, next_nid
                    ):
                        self.propagate(nid, next_nid)
                        self.worklist.append(next_nid)
                        self.visited.add(next_nid)

        return self.in_dict, self.out_dict


taint_analyzer = PossiblyTaintedDefinition()

In [ ]:
cfg = cfg_reader.read_cfg("../part_1/file_1.php.cfg.json")
taint = Taint.from_json("../part_1/file_1.php.taint.json")

In [ ]:
in_dict, out_dict = taint_analyzer(cfg, taint)
show_in_out(cfg, in_dict, out_dict)